~ Welcome to GAIA - Geospatial & Aerial Images Analyser [BETA] ~

*This program solution is not final and some details are potentially to be changed*

*New filters may be added to this program soon or later*

~ First steps to start ~

1) Open PATH.txt file in the directory of GAIA and past here the path to your folder with images which you want to analyze
    Ex: C:\\Users\\User\\Desktop\\GAIA\\IMAGES\\
    ! IT IS NECESSARY TO HAVE 2 BACK-SLASHES !

2) Start program using one of these ways:
    a) use start.bat file (it'll start Jupyter Notebook server on localhost:8888 and program will be opened in your browser)
    b) open GAIA.ipynb file in Jupyter Notebook IDE (DataSpell works well and program was tested in it)

3) Start the program using "Run All" button

4) Enter the name of image which you want to analyze

5) Get the result!

6) If you want to change the way some filter work - go to pre-config cell and change it however you want
    (original config file is located in the GAIA folder)

~ Included image filters ~

1) Gray filter
2) BGR filter
3) HSV filter
4) Blur filter
5) Contour filter
6) High-pass filter
7) Sepia filter
8) Denoising filter
9) Red, Green and Blue filters
10) Clahe filter
11) Component connectivity filter
12) Saturation inversion filter
13) Saturation filter
14) Hue filter
15) RGB filter
16) Cool filter
17) LWIR filter
18) SWIR filter
19) NDVI filter
20) Tasseled Cap filter
21) PCA filter
22) Transmission filter
23) Panchromatic filter
24) Inversion filter

NECESSARY IMPORTS

In [ ]:
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

IMAGE PATH

In [ ]:
path_file = open('PATH.txt', 'r+')
path_folder = path_file.read()
image_enter = input('Enter image name: ')
image_path = path_folder + image_enter
image = cv2.imread(image_path)

PRE-CONFIG

In [ ]:
#OUTPUT IMAGE SIZE
mpl.rcParams['figure.dpi'] = 500

#CONTOURS COLOUR (RGB)
R_color = 0
G_color = 255
B_color = 154

#GAUSSIAN BLUR COEFFICIENT
GC1 = GC2 = 1

#TASSELED CAP COEFFICIENTS
coefficients = [[0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863],
                [-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800],
                [0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572],
                [-0.8242, 0.0849, 0.4393, -0.0580, 0.2012, -0.2774],
                [-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085],
                [0.1084, -0.9022, 0.4119, -0.0575, 0.0029, -0.0251]]

#HIGH PASS FILTER KERNEL
hp_kernel = np.array([[0.0, -1.0, 0.0],
                   [-1.0, 4.0, -1.0],
                   [0.0, -1.0, 0.0]])

#SEPIA FILTER CONFIG
sepia_kernel = np.array([[0.272, 0.534, 0.131],[0.349, 0.686, 0.168],[0.393, 0.769, 0.189]])
ddepth = -1

#TRANSMISSION FILTER CONFIG
kernel_size = 15
atmospheric_light = np.percentile(image, 99)
transmission = 1 - 0.95 * (image.astype(np.float32) / atmospheric_light)

#PANCHROMATIC FILTER KERNEL
panchromatic_filter = np.array([[0.5, 1, 0.5],
                                [1, 1, 1],
                                [0.5, 1, 0.5]])

#CLAHE FILTER CONFIG
clipLimit = 5.0
tileGridSize = (8, 8)

#COMPONENT CONNECTIVITY CONFIG
thresh = 100
maxval = 255
thickness = 2
connectivity = 8

APPLYING RGB FILTER

In [ ]:
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

APPLYING GRAY FILTER

In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

APPLYING HSV FILTER

In [ ]:
image_hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

APPLYING BLURRING FILTER

In [ ]:
new_image = np.zeros(image.shape, dtype='uint8')
image_gray_blurred = cv2.GaussianBlur(image_gray, (GC1, GC2), 0)
image_gray_blurred_edges = cv2.Canny(image_gray_blurred, 100, 140)

APPLYING CONTOUR FILTER (ON BLACK SCREEN)

In [ ]:
contours, hierarchy = cv2.findContours(image_gray_blurred_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
contours, hir = cv2.findContours(image_gray_blurred_edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
image_contoured = cv2.drawContours(new_image, contours, -1, (R_color, G_color, B_color), 1)

APPLYING HIGH-PASS FILTER

In [ ]:
kernel = hp_kernel/(np.sum(hp_kernel) if np.sum(hp_kernel)!=0 else 1)
image_hp = cv2.filter2D(image,-1,kernel)

APPLYING SEPIA FILTER

In [ ]:
image_sepia = cv2.filter2D(src=image, kernel=sepia_kernel, ddepth=ddepth)

APPLYING DENOISING FILTER

In [ ]:
image_denoised = cv2.fastNlMeansDenoisingColored(image,None,20,10,7,21)

APPLYING COLOUR FILTERS

In [ ]:
#RED FILTERING
image_red = image.copy()
image_red[:, :, 0] = 0
image_red[:, :, 1] = 0

#GREEN FILTERING
image_green = image.copy()
image_green[:, :, 0] = 0
image_green[:, :, 2] = 0

#BLUE FILTERING
image_blue = image.copy()
image_blue[:, :, 1] = 0
image_blue[:, :, 2] = 0

APPLYING CLAHE FILTER

In [ ]:
clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
image_clahe = clahe.apply(image_gray)

APPLYING COMPONENT CONNECTIVITY FILTER

In [ ]:
image_ccp = image.copy()
_, binarized_img = cv2.threshold(image_gray, thresh=thresh, maxval=maxval, type=cv2.THRESH_BINARY)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binarized_img, connectivity=connectivity)

for i in range(1, num_labels):
    x, y, w, h, area = stats[i]
    rect_color = (0, 255, 0)
    image_cc = cv2.rectangle(image_ccp, (x, y), (x + w, y + h), rect_color, thickness=thickness)

APPLYING SATURATION INVERSION FILTER

In [ ]:
image_inv_sat = image_hsv.copy()
image_inv_sat[:, :, 1] = 255 - image_inv_sat[:, :, 1]
image_inv_sat = cv2.cvtColor(image_inv_sat, cv2.COLOR_HSV2BGR)

APPLYING SATURATION FILTER

In [ ]:
image_sat = image_hsv.copy()
image_sat[:, :, 0] = 0
image_sat[:, :, 2] = 255
image_sat = cv2.cvtColor(image_sat, cv2.COLOR_HSV2BGR)

APPLYING HUE FILTER

In [ ]:
image_hue = image_hsv.copy()
image_hue[:, :, 1] = 255
image_hue[:, :, 2] = 255
image_hue = cv2.cvtColor(image_hue, cv2.COLOR_HSV2BGR)

APPLYING RGB FILTER

In [ ]:
rgb_img = image.copy()
rgb_img[:, :, 0] = cv2.equalizeHist(image[:, :, 0])
rgb_img[:, :, 1] = cv2.equalizeHist(image[:, :, 1])
rgb_img[:, :, 2] = cv2.equalizeHist(image[:, :, 2])

APPLYING COOL FILTER

In [ ]:
img_cool = cv2.applyColorMap(image, cv2.COLORMAP_COOL)

APPLYING LWIR FILTER

In [ ]:
to_spectral = image_rgb.copy()
image_lwir = to_spectral[:, :, 2]

APPLYING SWIR FILTER

In [ ]:
image_swir = to_spectral[:, :, 1]

APPLYING NDVI FILTER

In [ ]:
to_ndvi = image_rgb.copy()

r, g, b = cv2.split(to_ndvi)

sum_img = g + r
mask = sum_img == 0
image_ndvi = np.zeros_like(g, dtype=np.float32)
image_ndvi[~mask] = (g - r)[~mask] / sum_img[~mask]

APPLYING TASSELED CAP TRANSFORMATION

In [ ]:
red = image[:, :, 0]
green = image[:, :, 1]
blue = image[:, :, 2]

tc_1 = coefficients[0][0] * blue + coefficients[0][1] * green + coefficients[0][2] * red + \
       coefficients[0][3] * 0 + coefficients[0][4] * 0 + coefficients[0][5] * 0

tc_2 = coefficients[1][0] * blue + coefficients[1][1] * green + coefficients[1][2] * red + \
       coefficients[1][3] * 0 + coefficients[1][4] * 0 + coefficients[1][5] * 0

tc_3 = coefficients[2][0] * blue + coefficients[2][1] * green + coefficients[2][2] * red + \
       coefficients[2][3] * 0 + coefficients[2][4] * 0 + coefficients[2][5] * 0

tc_4 = coefficients[3][0] * blue + coefficients[3][1] * green + coefficients[3][2] * red + \
       coefficients[3][3] * 0 + coefficients[3][4] * 0 + coefficients[3][5] * 0

tc_5 = coefficients[4][0] * blue + coefficients[4][1] * green + coefficients[4][2] * red + \
       coefficients[4][3] * 0 + coefficients[4][4] * 0 + coefficients[4][5] * 0

tc_6 = coefficients[5][0] * blue + coefficients[5][1] * green + coefficients[5][2] * red + \
       coefficients[5][3] * 0 + coefficients[5][4] * 0 + coefficients[5][5] * 0

tc_1_norm = (tc_1 - tc_1.min()) / (tc_1.max() - tc_1.min())
tc_2_norm = (tc_2 - tc_2.min()) / (tc_2.max() - tc_2.min())
tc_3_norm = (tc_3 - tc_3.min()) / (tc_3.max() - tc_3.min())
tc_4_norm = (tc_4 - tc_4.min()) / (tc_4.max() - tc_4.min())
tc_5_norm = (tc_5 - tc_5.min()) / (tc_5.max() - tc_5.min())
tc_6_norm = (tc_6 - tc_6.min()) / (tc_6.max() - tc_6.min())

APPLYING PCA FILTER

In [ ]:
to_pca = image_rgb.copy()
pixels = to_pca.reshape((-1, 3))
covariance = np.cov(pixels.T)
eigenvalues, eigenvectors = np.linalg.eig(covariance)
sort_indices = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[sort_indices]
eigenvectors = eigenvectors[:, sort_indices]
first_pc = eigenvectors[:, 0]
second_pc = eigenvectors[:, 1]
projection = np.dot(pixels, eigenvectors)
channel1 = projection[:, 0].reshape(to_pca.shape[0], to_pca.shape[1])
channel2 = projection[:, 1].reshape(to_pca.shape[0], to_pca.shape[1])

APPLYING TRANSMISSION FILTER

In [ ]:
if image.dtype != np.uint8:
    image = image.astype(np.uint8)
if image.ndim == 3 and image.shape[-1] not in [1, 3, 4]:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

transmission = (cv2.medianBlur((255*transmission).astype(np.uint8), kernel_size)).astype(np.float32) / 255

APPLYING PANCHROMATIC FILTER

In [ ]:
filtered_img = cv2.filter2D(image, -1, panchromatic_filter)

APPLYING INVERSION FILTER

In [ ]:
image_inv = cv2.bitwise_not(image)

SHOW ORIGINAL IMAGE

In [ ]:
plt.imshow(image_rgb)
plt.title('ORIGINAL IMAGE')
plt.show()

SHOW IMAGE IN RGB

In [ ]:
plt.imshow(image)
plt.title('BGR FILTER')
plt.show()

SHOW GRAY IMAGE

In [ ]:
plt.imshow(image_gray)
plt.title('GRAY FILTER')
plt.show()

SHOW IMAGE IN HSV

In [ ]:
plt.imshow(image_hsv)
plt.title('HSV FILTER')
plt.show()

SHOW CONTOURED IMAGE ON BLACK SCREEN

In [ ]:
plt.imshow(image_contoured)
plt.title('CONTOURING')
plt.show()

SHOW HIGH PASS FILTERED BLACK SCREEN IMAGE

In [ ]:
plt.imshow(image_hp)
plt.title('HIGH-PASS FILTER')
plt.show()

SHOW SEPIA FILTERED IMAGE

In [ ]:
plt.imshow(image_sepia, cmap="gray")
plt.title('SEPIA FILTER')
plt.show()

SHOW DENOISED IMAGE

In [ ]:
plt.imshow(image_denoised)
plt.title('DENOISING FILTER')
plt.show()

SHOW RED-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_red, cv2.COLOR_BGR2RGB))
plt.title('RED FILTER')
plt.show()

SHOW GREEN-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_green, cv2.COLOR_BGR2RGB))
plt.title('GREEN FILTER')
plt.show()

SHOW BLUE-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_blue, cv2.COLOR_BGR2RGB))
plt.title('BLUE FILTER')
plt.show()

SHOW SATURATION-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_sat, cv2.COLOR_BGR2RGB))
plt.title('SATURATION FILTER')
plt.show()

SHOW HUE-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_hue, cv2.COLOR_BGR2RGB))
plt.title('HUE FILTER')
plt.show()

SHOW INVERSION-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_inv, cv2.COLOR_BGR2RGB))
plt.title('INVERSION FILTER')
plt.show()

SHOW RGB IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB))
plt.title('RGB FILTER')
plt.show()

SHOW SATURATION INVERSION FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_inv_sat, cv2.COLOR_BGR2RGB))
plt.title('SATURATION INVERSION FILTER')
plt.show()

SHOW CLAHE-FILTERED IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_clahe, cv2.COLOR_GRAY2RGB))
plt.title('CLAHE FILTER')
plt.show()

SHOW COMPONENT CONNECTIVITY ON ORIGINAL IMAGE

In [ ]:
plt.imshow(cv2.cvtColor(image_cc, cv2.COLOR_BGR2RGB))
plt.title('COMPONENT CONNECTIVITY FILTER (ORIGINAL IMAGE)')
plt.show()

SHOW COMPONENT CONNECTIVITY ON MONOTONOUS BACKGROUND

In [ ]:
plt.imshow(labels, cmap='jet')
plt.title('COMPONENT CONNECTIVITY FILTER (MONOTONOUS BACKGROUND)')
plt.show()

SHOW COOL FILTERED IMAGE

In [ ]:
plt.imshow(img_cool)
plt.title('COOL FILTER')
plt.show()

SHOW NDVI FILTERED IMAGE

In [ ]:
plt.imshow(image_ndvi, cmap='jet')
plt.title('NDVI FILTER')
plt.show()

SHOW SWIR FILTERED IMAGE

In [ ]:
plt.imshow(image_swir) #cmap='gray'
plt.title('SWIR FILTER')
plt.show()

SHOW LWIR FILTERED IMAGE

In [ ]:
plt.imshow(image_lwir)  #cmap gray
plt.title('LWIR FILTER')
plt.show()

SHOW PCA FILTERED IMAGE

In [ ]:
plt.imshow(channel1) #cmap='gray'
plt.title('PCA FILTER')
plt.show()

SHOW PANCHROMATIC FILTERED IMAGE

In [ ]:
plt.imshow(filtered_img)
plt.title('PANCHROMATIC FILTER')
plt.show()

SHOW TRANSMISSION FILTERED IMAGE

In [ ]:
plt.imshow(transmission)
plt.title('TRANSMISSION FILTER')
plt.show()

SHOW TASSELED CAP FILTERED IMAGE

In [ ]:
plt.imshow(tc_1_norm)
plt.title('TASSELED CAP FILTER (1)')
plt.show()

In [ ]:
plt.imshow(tc_2_norm)
plt.title('TASSELED CAP FILTER (2)')
plt.show()

In [ ]:
plt.imshow(tc_3_norm)
plt.title('TASSELED CAP FILTER (3)')
plt.show()

In [ ]:
plt.imshow(tc_4_norm)
plt.title('TASSELED CAP FILTER (4)')
plt.show()

In [ ]:
plt.imshow(tc_5_norm)
plt.title('TASSELED CAP FILTER (5)')
plt.show()

In [ ]:
plt.imshow(tc_6_norm)
plt.title('TASSELED CAP FILTER (6)')
plt.show()